In [97]:
import numpy as np
from bposd.hgp import hgp

"""
Import our (3,4) code --> map to Joschka --> run BP --> map output to [vv_error, cc_error] --> run ssf with modified score.
"""

classical_code='mkmn_16_4_6.txt'
seed_code = np.loadtxt(f"examples/codes/classical_seed_codes/{classical_code}").astype(int)
    # print(seed_code)
qcode = hgp(seed_code,compute_distance=True)

qcode.test(), seed_code

<Unnamed CSS code>, (4,7)-[[400,16,6]]
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params (4,7)-[[400,16,6]]


(True,
 array([[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
        [0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
        [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
        [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
        [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]]))

In [58]:
from ldpc import bp_decoder
MAX_ITERS = 20
ERROR_RATE = 0.05
# Use the bp decoder

bpd = bp_decoder(qcode.hz,  # the parity check matrix
	error_rate=ERROR_RATE,
	max_iter=MAX_ITERS,
	bp_method="ps"
)


In [85]:
error=np.random.choice(2, qcode.N, p=[1 - ERROR_RATE, ERROR_RATE])
syndrome=qcode.hz@error % 2
guessed_error = bpd.decode(syndrome)
residual_error = (error + guessed_error) % 2
sum(guessed_error), sum(qcode.hz@residual_error% 2), sum(syndrome)

(16, 4, 43)

In [ ]:
"""
	Here we assume that the code is generating from an hgp with 1 H1 x H1
"""
def get_vv_cc_residual_errors(errors, code):
	# n^2 vv bits and m ** 2 cc bits
	vv_bits = code.n1 ** 2
	return errors[:vv_bits], error[vv_bits:]
get_vv_cc_residual_errors(residual_error, qcode)

## Running on our 3,4 codes

In [109]:
import sys
import os

from src.ssf.read_ccode import Classical_code, read_ccode
"""
	Convert sparse code representation to a matrix layout
"""
def convert_ccode_to_joschka_ccode(ccode: Classical_code):
	H = np.zeros([ccode.m, ccode.n]).astype(int)
	# c is the checks
	for c in range(ccode.m):
		for v in ccode.check_nbhd[c]:
			H[c][v] = 1
	return H

code_3_4 = read_ccode(['examples/codes/ssf_code/80_60.code'], [], [], [], [], [])[0]
code_3_4_josh = convert_ccode_to_joschka_ccode(code_3_4)
qcode = hgp(code_3_4_josh,compute_distance=True)